In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import importlib
import transformer_type
from transformer_type.dataset import get_data
import torch
import pandas as pd

if torch.cuda.is_available(): device = torch.device ("cuda")
elif torch.backends.mps.is_available(): device = torch.device("mps")
else: device = torch.device('cpu')

In [2]:
# params = [120, 1, 64, 300, 32, 512, 2, 1e-5, 0.1]
params = {'input_window' : [120],
         'pred_step' : [60, 90, 120, 150],
         'batch_size' : [32],
         'epochs' : [100],
         'feature_size' : [32],
         'd_ff' : [512],
         'num_layers' : [2],
         'lr' : [1e-4],
         'device' : [device],
         'dropout' : [0.1],
         'is_save' : [True]}

In [3]:
DATA_PATH = "./data/kaggle_data/daily_data/"

In [4]:

# make dataset

def data_extract(df_dict, df, name, cols):
       for col in cols:
              df_dict[f"{name}_{col}"] = df.loc[df[col] != 0, col]
       return df_dict

train_data_list = []
val_data_list = []


# rom_df = pd.read_csv(DATA_PATH + "romania_elec_daily.csv")
# rom_cols = ['Consumption']
pmj_df = pd.read_csv(DATA_PATH + "pjm_elec_daily.csv")
pmj_cols = ['AEP', 'COMED', 'DAYTON', 'DOM', 'DUQ', 'FE', 'NI', 'PJME', 'PJMW'] # 'DEOK', 'EKPC', 'PJM_Load'

# ind_df = pd.read_csv(DATA_PATH + "india_elec_daily.csv")
# ind_cols = ['Punjab', 'Haryana', 'Rajasthan', 'Delhi', 'UP',
#        'Uttarakhand', 'HP', 'J&K', 'Chandigarh', 'Chhattisgarh', 'Gujarat',
#        'MP', 'Maharashtra', 'Goa', 'DNH', 'Andhra Pradesh', 'Telangana',
#        'Karnataka', 'Kerala', 'Tamil Nadu', 'Pondy', 'Bihar', 'Jharkhand',
#        'Odisha', 'West Bengal', 'Sikkim', 'Arunachal Pradesh', 'Assam',
#        'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Tripura']
# mal_df = pd.read_csv(DATA_PATH + "malaysia_elec_daily.csv")
# mal_cols = ['load']
# tet_df = pd.read_csv(DATA_PATH + "tetouan_elec_daily.csv")
# tet_cols = ['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']
# steel_df = pd.read_csv(DATA_PATH + "steel_industry_elec_daily.csv")
# steel_cols = ['Usage_kWh']

df_dict = {}

# df_dict = data_extract(df_dict, rom_df, 'ROM', rom_cols)
df_dict = data_extract(df_dict, pmj_df, 'PMJ', pmj_cols)

# df_dict = data_extract(df_dict, mal_df, 'MAL', mal_cols)
# df_dict = data_extract(df_dict, ind_df, 'IND', ind_cols)
# df_dict = data_extract(df_dict, tet_df, 'TET', tet_cols)
# df_dict = data_extract(df_dict, steel_df, 'STEEL', steel_cols)

for df_name in df_dict:
    df = df_dict[df_name]
    train_data, val_data, scaler = get_data(df, params['input_window'][0], 1)
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    print(df_name, len(train_data), len(val_data))
    train_data_list.append((train_data, df_name, scaler))
    val_data_list.append((val_data, df_name, scaler))



PMJ_AEP 3922 889
PMJ_COMED 2095 433
PMJ_DAYTON 3922 889
PMJ_DOM 3752 847
PMJ_DUQ 3848 871
PMJ_FE 1974 403
PMJ_NI 1827 366
PMJ_PJME 4725 1090
PMJ_PJMW 4653 1072


In [5]:
cnt = 0
total = 1
best_models, models = None, None
for param in params:
    total *= len(params[param])


def select_param(params, param_names, selected_params=[], param_idx=0):
    if param_idx == len(params):
        global best_models, models
        importlib.reload(transformer_type)
        from transformer_type.transformer_multistep_type import train_start
        torch.cuda.empty_cache()
        global cnt
        cnt += 1
        print(f"Start training ({cnt} / {total})")
        for param_name, params in zip(param_names, selected_params):
            print(f"{param_name} : {params}", end= ' | ')
        print()
        selected_params = [train_data_list, val_data_list] + selected_params
        best_models, models = train_start(*selected_params)
        return
    
    param_name = param_names[param_idx]
    param_list = params[param_name]
    for parameter in param_list:
        select_param(params, param_names, selected_params + [parameter], param_idx + 1)


In [6]:
select_param(params, list(params.keys()))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Start training (1 / 4)
input_window : 120 | pred_step : 60 | batch_size : 32 | epochs : 100 | feature_size : 32 | d_ff : 512 | num_layers : 2 | lr : 0.0001 | device : cuda | dropout : 0.1 | is_save : True | 


wandb: Currently logged in as: qja1998. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
100%|██████████| 100/100 [12:07<00:00,  7.28s/it]

Start training (2 / 4)
input_window : 120 | pred_step : 90 | batch_size : 32 | epochs : 100 | feature_size : 32 | d_ff : 512 | num_layers : 2 | lr : 0.0001 | device : cuda | dropout : 0.1 | is_save : True | 


...

pred_MAE,▁▁▂▄▅▆▇▇██
pred_MSE,▁▁▃▄▅▅▆▇▇█
pred_SMAPE,█▇▄▃▂▂▁▁▁▁
val_loss_PMJ_AEP,████▆▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_COMED,██▇████▇▇▆▄▄▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
val_loss_PMJ_DAYTON,████▇▆▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_DOM,██▇▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_DUQ,█████▇▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_FE,█▇▇▆▆▆▆▅▅▄▅▆▅▅▅▆▄▄▅▅▄▃▄▄▅▄▅▄▂▃▃▄▂▂▃▂▂▁▂▁
val_loss_PMJ_NI,▇▇████▇▇█▇▇▇▆▆▅▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_PJME,▇██▇▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


100%|██████████| 100/100 [12:28<00:00,  7.48s/it]

Start training (4 / 4)
input_window : 120 | pred_step : 150 | batch_size : 32 | epochs : 100 | feature_size : 32 | d_ff : 512 | num_layers : 2 | lr : 0.0001 | device : cuda | dropout : 0.1 | is_save : True | 


pred_MAE,▁▂▃▄▅▅▆▇██
pred_MSE,▁▂▃▄▅▅▆▇▇█
pred_SMAPE,▁▅▆▇████▇▇
val_loss_PMJ_AEP,▇█▇█▇▅▄▄▄▄▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_COMED,████▇▇▇▇▇▇▆▆▆▆▇▆▆▆▆▆▆▅▅▄▄▄▄▄▄▃▄▄▃▃▂▂▂▁▂▁
val_loss_PMJ_DAYTON,█████▇▇▆▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_DOM,███▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_DUQ,█▇█▇▆▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_FE,█▇▇▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
val_loss_PMJ_NI,▇███▇▇▇▇▇▇▆▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss_PMJ_PJME,███▆▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


100%|██████████| 100/100 [12:25<00:00,  7.46s/it]
